In [5]:
import subprocess


def extract_audio_from_video(video_path, audio_path):
    command = [
        "ffmpeg",
        "-i",
        video_path,
        "-vn",
        audio_path,
    ]
    subprocess.run(command)


extract_audio_from_video(
    "podcast.mp4",
)

[youtube] Extracting URL: https://www.youtube.com/watch?v=K3xJ7GQuHpw
[youtube] K3xJ7GQuHpw: Downloading webpage
[youtube] K3xJ7GQuHpw: Downloading ios player API JSON
[youtube] K3xJ7GQuHpw: Downloading web creator player API JSON
[youtube] K3xJ7GQuHpw: Downloading player 37b24ccb
[youtube] K3xJ7GQuHpw: Downloading m3u8 information
[info] K3xJ7GQuHpw: Downloading 1 format(s): 616+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 306
[download] Destination: c:\Users\dioxin\Desktop\course\FULLSTACK-GPT\1.f616.mp4
[download] 100% of  117.79MiB in 00:01:11 at 1.64MiB/s                   
[download] Destination: c:\Users\dioxin\Desktop\course\FULLSTACK-GPT\1.f140.m4a
[download] 100% of   27.25MiB in 00:00:04 at 6.04MiB/s     
[Merger] Merging formats into "c:\Users\dioxin\Desktop\course\FULLSTACK-GPT\1.mp4"
Deleting original file c:\Users\dioxin\Desktop\course\FULLSTACK-GPT\1.f616.mp4 (pass -k to keep)
Deleting original file c:\Users\dioxin\Desktop\course\FULLSTACK-GPT\